In [2]:
import pandas as pd
from matplotlib import pyplot as plt
from copy import deepcopy
import numpy as np
import random 
from datetime import datetime
from sklearn.preprocessing import StandardScaler

def print_data_summary(data):
    print("There are {0} datapoints".format(len(data)))
    print("There are {0} columns".format(len(data.columns)))
    try:
        print("There are {0} diffenret movies".format(len(set(data['s_video_id']))))
    except:
        pass
    # print(data.describe())
    
def encode_one_hot(data):
    categorical_columns = get_categorical_columns(data)
    oneHotEncodedData = pd.get_dummies(data, columns=categorical_columns)
    
    # Test
    newNumberOfColumns = len(oneHotEncodedData.columns) 

    totalDifferentValues = 0
    for c in categorical_columns:
        totalDifferentValues += len(data[c].unique())

    # One hot encoding removes each categorical columns (- len(nonNumericalList)) 
    # and adds n columns where n is number of different values taken by column (+ totalDifferentValues)
    assert len(data.columns) + totalDifferentValues - len(categorical_columns) == newNumberOfColumns
    
    return oneHotEncodedData

def split_train_test(data, test_mask):
    # split between train and test
    train_mask = list(map(lambda b: not b, test_mask))

    train = data[train_mask]
    test = data[test_mask]

    assert len(train) + len(test) == len(data) 

    y_train = train.loc[:, ['t_average_vmaf']]
    x_train = train.drop(['t_average_vmaf'], axis=1)
    assert type(x_train) == type(y_train) 

    y_test = test.loc[:, ['t_average_vmaf']]
    x_test = test.drop(['t_average_vmaf'], axis=1)
    assert type(x_test) == type(y_test)

    assert len(x_train) == len(y_train)
    assert len(x_train.columns) + len(y_train.columns) == len(data.columns)
    assert len(x_test) == len(y_test)
    assert len(x_test.columns) + len(y_test.columns) == len(data.columns)
    return x_train, y_train, x_test, y_test

def calculate_rmse(y_true, y_predictions):
    mse = mean_squared_error(y_true, y_predictions)
    rmse = math.sqrt(mse)
    return rmse

def calculate_std(y_true, y_predictions):
    '''
    Standard deviation is calculated considering the absoulte values of the residuals as the
    values and the RMSE as the mean.
    '''
    rmse = calculate_rmse(y_true, y_predictions)
    
    abs_residuals = np.absolute(y_true - y_predictions)
    
    
    #print(abs_residuals)
    #print(abs_residuals.tolist().sort(reverse=True))
    tbp = list(np.ravel(abs_residuals))
    tbp.sort(reverse=True)
    print("Printing the 4th quartile to doublecheck std: ")
    print(np.percentile(tbp, 75))  # Q3
    #print(tbp[0:20])

    
    rmse_column = np.array([rmse]*len(abs_residuals)).reshape(len(abs_residuals), 1)

    #print("len(abs_residuals)")
    #print(len(abs_residuals))
    #print("rmse_column")
    #print(rmse_column)
    variance = np.sum(np.square(abs_residuals - rmse_column)) / len(abs_residuals)
    std = math.sqrt(variance)
    return std

def run_model(regression_model, x_train, y_train, x_test, y_test):
    '''
    Given a sklearn model and the train / test data,
    returns the rmse, std and r squared scores of the model
    also returns training time in seconds
    '''
    start_time = datetime.now().timestamp()
   
    regular_regression = regression_model.fit(x_train.to_numpy(), y_train.to_numpy())

    end_time = datetime.now().timestamp()
   
    predictions = regression_model.predict(x_test.to_numpy())

    rmse = calculate_rmse(y_test.to_numpy(), predictions)

    std = calculate_std(y_test.to_numpy(), predictions)

    coefficient_of_determination = regression_model.score(x_test.to_numpy(), y_test.to_numpy())
    
    return Model_results(rmse, std, coefficient_of_determination, end_time - start_time, regression_model)

def analyse_model(regression_model, x_train, y_train, x_test, y_test):
    regular_regression = regression_model.fit(x_train.to_numpy(), y_train.to_numpy())
    
    paramRelevance = {'Parameter':x_train.columns.to_list()[:],
            'Weight':regular_regression.coef_.tolist()[0]}
 
    dfParam = pd.DataFrame(paramRelevance)

    print(dfParam.sort_values(by='Weight', ascending=False)[:])

class Model_results:
    def __init__(self, rmse, std, cod, time, model):
        self.rmse = rmse
        self.std = std
        self.cod = cod
        self.time = time
        self.model = model
        
    def __str__(self):
        attrs = vars(self)
        return ', '.join("%s: %s" % item for item in attrs.items())
    
def get_numerical_columns(data):
    numerical = data.select_dtypes(exclude=['object'])
    exclude_categorical = [col for col in numerical.columns \
                                   if not 'content_category' in col \
                                   and not 'scan_type' in col \
                                   and not 'codec_profile' in col \
                                   and not 's_video_id' in col]
    return exclude_categorical

def get_categorical_columns(data):
    numerical_columns = set(get_numerical_columns(data))
    return set(data.columns) - numerical_columns

def explore_categorical_features(data):
    # explorig categorical columns
    nonNumericalFrame = data.select_dtypes(include=['object']).copy()
    nonNumericalList = nonNumericalFrame.columns.tolist()
    totalDifferentValues = 0
    for c in nonNumericalList:
        print("{0} is non-numerical and values are: {1}".format(c, nonNumericalFrame[c].unique()))
        totalDifferentValues += len(nonNumericalFrame[c].unique())

    print("There are {0} total possible values among the categorical columns".format(totalDifferentValues))

    # content_category is the most problematic column because it has many possible values
    # nonNumericalFrame["c_content_category"].value_counts()

In [3]:
data = pd.read_csv("data.csv")
rawColumns = data.columns
rawData = deepcopy(data)

# test 
assert len(get_numerical_columns(data)) + len(get_categorical_columns(data)) == len(data.columns)

In [4]:
# PREPROCESSING

# 1 Remove empty columns
data = data.dropna(axis=1, thresh=len(data.columns))
print("The following columns have been removed because empty: {0}".format(set(rawColumns) - set(data.columns)))

# 2 Remove rows with missing values
threshold = len(data.columns)
data = data.dropna(axis=0, thresh=threshold)
print("There were {0} rows deleted because they had less than {1} valorised features".format(len(rawData) - len(data), threshold))

# 3 Remove columns that always have the same value
nunique = data.apply(pd.Series.nunique)
cols_to_drop = nunique[nunique == 1].index
data = data.drop(cols_to_drop, axis=1)
for c in cols_to_drop.tolist():
    print("Column {0} always has same value: {1}, removed".format(c, rawData[c].tolist()[1]))
assert len(nunique) == len(data.columns) + len(cols_to_drop)

# 4 Remove video id
data = data.drop(['s_video_id'], axis = 1)

print("\nAfter inital processing")
print_data_summary(data)

# 5 Coalesce width and height to a categorical feature
# print(data.groupby(['s_width','s_height']).size().reset_index().rename(columns={0:'count'}))

data['s_size'] = data.apply(lambda row: str(row['s_width']) + "_" + str(row['s_height']), axis=1)
data = data.drop(['s_width', 's_height'], axis = 1)

# 6 convert size to 

# explore_categorical_features(data)

# 6 Scale data (mean is zero)

# split data in numerical, categorical, y columns
y = data['t_average_vmaf']
categorical = data[get_categorical_columns(data.drop(['t_average_vmaf'], axis = 1))]
numerical = data[get_numerical_columns(data.drop(['t_average_vmaf'], axis = 1))]

# scale only numerical
scaler = StandardScaler()
scaler.fit(numerical)
scaled_numerical = scaler.transform(numerical)

# recombine everything
numerical_data = pd.DataFrame(scaled_numerical, numerical.index, columns=numerical.columns)
# test
assert len(numerical_data.columns) + len(categorical.columns) + 1 == len(data.columns)
data = pd.concat([numerical_data, categorical, y], axis=1)

print("\nAfter scaling")
print_data_summary(data)

# 5 One hot encoding categorical columns

# pro wrt label encoding: the model wont derive false relations between columns based on numerical values (2 > 1)
# con wrt label encoding: more columns are introduced

data = encode_one_hot(data)
print("\nAfter 1 hot encoding categorical features")
print_data_summary(data)


preprocessed_data = deepcopy(data)

preprocessed_data.head()

The following columns have been removed because empty: {'t_average_vmaf_mobile', 't_average_vmaf_4k', 't_average_psnr'}
There were 2883 rows deleted because they had less than 43 valorised features
Column e_aspect_ratio always has same value: 16:09, removed
Column e_pixel_aspect_ratio always has same value: 1:01, removed
Column e_codec always has same value: h264, removed
Column e_b_frame_int always has same value: 3, removed
Column e_ref_frame_count always has same value: 1.0, removed
Column e_bit_depth always has same value: 8.0, removed
Column e_pixel_fmt always has same value: yuv420p, removed

After inital processing
There are 12133 datapoints
There are 35 columns

After scaling
There are 12133 datapoints
There are 34 columns

After 1 hot encoding categorical features
There are 12133 datapoints
There are 110 columns


,s_storage_size,s_duration,c_si,c_ti,c_scene_change_ffmpeg_ratio30,c_scene_change_ffmpeg_ratio60,c_scene_change_ffmpeg_ratio90,c_scene_change_py_thresh30,c_scene_change_py_thresh50,c_colorhistogram_mean_dark,...,e_codec_profile_high,e_codec_profile_main,s_size_1080_1080,s_size_1280_720,s_size_1920_1080,s_size_1920_800,s_size_1920_818,s_size_352_288,s_size_720_486,s_size_720_608
0,-0.49794,-0.598653,0.653567,-0.41539,-0.239026,-0.74125,-0.512497,-1.518056,-1.161942,-0.214322,...,1,0,0,0,1,0,0,0,0,0
1,-0.49794,-0.598653,0.653567,-0.41539,-0.239026,-0.74125,-0.512497,-1.518056,-1.161942,-0.214322,...,0,1,0,0,1,0,0,0,0,0
2,-0.49794,-0.598653,0.653567,-0.41539,-0.239026,-0.74125,-0.512497,-1.518056,-1.161942,-0.214322,...,0,1,0,0,1,0,0,0,0,0
3,-0.49794,-0.598653,0.653567,-0.41539,-0.239026,-0.74125,-0.512497,-1.518056,-1.161942,-0.214322,...,0,1,0,0,1,0,0,0,0,0
4,-0.49794,-0.598653,0.653567,-0.41539,-0.239026,-0.74125,-0.512497,-1.518056,-1.161942,-0.214322,...,0,1,0,0,1,0,0,0,0,0


In [5]:
# Data augmentation

# from [x, y, z] we want to obtain 
# [x, y, z, x**2, y**2, z**2, x*y, x*z, y*z]
# aka all the degree 2 polinomyals 

# only consider numerical columns, exluce categorical

exclude_categorical = get_numerical_columns(preprocessed_data)

theoretical_final_number_of_columns = len(data.columns) + \
+ len(exclude_categorical) + \
((len(exclude_categorical)) * (len(exclude_categorical) - 1)) / 2

baseForAugmentation = data[exclude_categorical]

columnsCopy = baseForAugmentation.columns.tolist()

print("original oclumns {0}".format(len(baseForAugmentation.columns)))

for c in baseForAugmentation.columns:

    data[c + "_squared"] = data[c] ** 2
    data[c + "cubic"] = data[c] ** 3
    
    columnsCopy.remove(c)
    for cc in columnsCopy:
        t = data[c] * data[cc]
        data[c + "_*_" + cc ] = data[c] * data[cc]

# assert(int(theoretical_final_number_of_columns) == len(data.columns.tolist()))

print("After augmenting the data")
print_data_summary(data)
augmented_data = deepcopy(data)
augmented_data.head()

original oclumns 37
After augmenting the data
There are 12133 datapoints
There are 850 columns


,s_storage_size,s_duration,c_si,c_ti,c_scene_change_ffmpeg_ratio30,c_scene_change_ffmpeg_ratio60,c_scene_change_ffmpeg_ratio90,c_scene_change_py_thresh30,c_scene_change_py_thresh50,c_colorhistogram_mean_dark,...,s_size_1920_818_*_s_size_720_608,s_size_352_288_squared,s_size_352_288cubic,s_size_352_288_*_s_size_720_486,s_size_352_288_*_s_size_720_608,s_size_720_486_squared,s_size_720_486cubic,s_size_720_486_*_s_size_720_608,s_size_720_608_squared,s_size_720_608cubic
0,-0.49794,-0.598653,0.653567,-0.41539,-0.239026,-0.74125,-0.512497,-1.518056,-1.161942,-0.214322,...,0,0,0,0,0,0,0,0,0,0
1,-0.49794,-0.598653,0.653567,-0.41539,-0.239026,-0.74125,-0.512497,-1.518056,-1.161942,-0.214322,...,0,0,0,0,0,0,0,0,0,0
2,-0.49794,-0.598653,0.653567,-0.41539,-0.239026,-0.74125,-0.512497,-1.518056,-1.161942,-0.214322,...,0,0,0,0,0,0,0,0,0,0
3,-0.49794,-0.598653,0.653567,-0.41539,-0.239026,-0.74125,-0.512497,-1.518056,-1.161942,-0.214322,...,0,0,0,0,0,0,0,0,0,0
4,-0.49794,-0.598653,0.653567,-0.41539,-0.239026,-0.74125,-0.512497,-1.518056,-1.161942,-0.214322,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# PCA
from sklearn.decomposition import PCA
import math

# for PCA to work well data needs to be scaled (mean is zero)
scaler = StandardScaler()
scaler.fit(augmented_data.drop(['t_average_vmaf'], axis=1))
scaledData = scaler.transform(augmented_data.drop(['t_average_vmaf'], axis=1))

# reducing the data to an arbitrary number of dimensions
pca = PCA(n_components=5)
pca.fit(scaledData)
pca_data = pca.transform(scaledData)

# Principal components are new variables that are constructed 
# as linear combinations or mixtures of the initial variables.
# We don't have the original headers names

pca_data = pd.DataFrame(data=pca_data)

if not pca_data.index.equals(augmented_data.index):
    pca_data.index = augmented_data.index
    
# adding the y back
# it was removed because otherwise it would have been lost in the pca process
pca_data['t_average_vmaf'] = augmented_data['t_average_vmaf']

print("After applying PCA to the data")
print_data_summary(pca_data)

After applying PCA to the data
There are 12133 datapoints
There are 6 columns


In [7]:
# Splitthe 3 datasets (preprocessed, augmented, PCAed)
# Note the the same splitting is applied across all 3 

test_mask = np.random.rand(len(data), 1) > 0.60

x_train_pre, y_train_pre, x_test_pre, y_test_pre = split_train_test(preprocessed_data, test_mask)
x_train_aug, y_train_aug, x_test_aug, y_test_aug = split_train_test(augmented_data, test_mask)
x_train_pca, y_train_pca, x_test_pca, y_test_pca = split_train_test(pca_data, test_mask)

In [10]:
from keras.models import Sequential
from keras.layers import Dense, Input, Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from keras.utils import plot_model
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import pickle

# given nn parameters, create a nn with those parameters
# nn1 is for single layer
# nn2 is for 2 layers
def make_nn1_closure(firstLayerNodes, dropOutPercentage, optimiser, activation, input_dim):
    def make_nn1():
        model = Sequential()
        model.add(Dense(firstLayerNodes, activation=activation, input_dim=input_dim))
        model.add(Dropout(dropOutPercentage))
        model.add(Dense(1, kernel_initializer='normal'))
        # plot_model(model, show_shapes=True, rankdir="LR")
        model.compile(loss='mean_squared_error', optimizer=optimiser)
        return model
    return make_nn1

def make_nn2_closure(firstLayerNodes, secondLayerNodes, dropOutPercentage, optimiser, activation, input_dim):
    def make_nn2():
        model = Sequential()
        model.add(Dense(firstLayerNodes, activation=activation, input_dim=input_dim))
        model.add(Dropout(dropOutPercentage))
        model.add(Dense(secondLayerNodes, activation=activation))
        model.add(Dense(1, kernel_initializer='normal'))
        # plot_model(model, show_shapes=True, rankdir="LR")
        model.compile(loss='mean_squared_error', optimizer=optimiser)
        return model
    return make_nn2

# Class that encapsulates all the different parameters we search for
class Params:
        
        def __init__(self, first_nodes, second_nodes, epochs, kfold, opti, acti, drop, batch_size):
            self.first_nodes = first_nodes
            self.second_nodes = second_nodes
            self.epochs = epochs
            self.kfold = kfold
            self.optimizer = opti
            self.activation = acti
            self.drop = drop
            self.batch_size = batch_size
            
        def __str__(self):
            attrs = vars(self)
            return ', '.join("%s: %s" % item for item in attrs.items())
        
# Class that represent the search space (ie the grid) for the parameters
class Grid_parameters:
    nodeValues = list(range(5, 50))
    batchValues = [2, 4, 8, 16, 32]
    epochesValues = [2]#list(range(35, 75, 10))
    kFoldSplitValues = [10, 5, 12]
    optimizers = ["RMSprop", "adam", "Adadelta", "Nadam"]
    activations=["softmax", "relu", "tanh"]
    dropOuts = list(np.arange(0.0, 0.8, 0.05))
    
    def __init__(self):
        pass
    
    def get_random_params(self):
        return Params(random.choice(self.nodeValues), 
                      random.choice(self.nodeValues),
                      random.choice(self.epochesValues),
                      random.choice(self.kFoldSplitValues),
                      random.choice(self.optimizers),
                      random.choice(self.activations),
                      random.choice(self.dropOuts),
                      random.choice(self.batchValues)
                     )

ModuleNotFoundError: No module named 'keras'

In [ ]:
## Uncomment for intermediary results
# PICKLE READ FILE

#infile = open('results_pre','rb')
#new_dict = pickle.load(infile)
#print(new_dict[0][0])
#infile.close()

grid = Grid_parameters()

results_pre = []
results_aug = []
results_pca = []

for i in range(0, 50):
    # for 50 times get a random nn topology from the grid and get cross validaiton results for
    # normal, augmented and PCAed data
    params = grid.get_random_params()
    print(params)
    
    estimators_1 = []
    estimators_2 = []
    estimators_1.append(('standardize', StandardScaler()))
    estimators_2.append(('standardize', StandardScaler()))
    
    model_1 = make_nn1_closure(params.first_nodes, params.drop, params.optimizer, params.activation, x_train_pre.shape[1])
    model_2 = make_nn2_closure(params.first_nodes, params.second_nodes, params.drop, params.optimizer, params.activation, x_train_pre.shape[1])
    
    estimators_1.append(('mlp', KerasRegressor(build_fn=model_1, 
                                             epochs=params.epochs, 
                                             batch_size=params.batch_size, 
                                             verbose=1)))
    estimators_2.append(('mlp', KerasRegressor(build_fn=model_2, 
                                             epochs=params.epochs, 
                                             batch_size=params.batch_size, 
                                             verbose=1)))
    
    pipeline_1 = Pipeline(estimators_1)
    pipeline_1 = Pipeline(estimators_2)
    
    kfold = KFold(n_splits=params.kfold)
    
    result_pre_1 = cross_val_score(pipeline_1, x_train_pre.to_numpy(), y_train_pre.to_numpy(), cv=kfold)
    result_pre_2 = cross_val_score(pipeline_2, x_train_pre.to_numpy(), y_train_pre.to_numpy(), cv=kfold)
    #result_aug = cross_val_score(pipeline, x_train_pre.to_numpy(), y_train_pre.to_numpy(), cv=kfold)
    #result_pca = cross_val_score(pipeline, x_train_pre.to_numpy(), y_train_pre.to_numpy(), cv=kfold)
    
    print("Standardized: mean is %.2f, std is %.2f" % (result_pre.mean(), result_pre.std()))
    #print("Standardized: mean is %.2f, std is %.2f" % (result_aug.mean(), result_aug.std()))
    #print("Standardized: mean is %.2f, std is %.2f" % (result_pca.mean(), result_pca.std()))
    
    results_pre.append((params, result_pre))
    #results_aug.append((params, result_aug))
    #results_pca.append((params, result_pca))
    
    # Save results to file
    
    with open('results_pre', 'wb') as save:
        pickle.dump(results_pre, save)
    
    
    
# below I copied pased the best results 

# No Dropout
# Standardized: mean is -46.98, std is 18.11
            # running model with 29 firstLayerNodes, 8 secondLayerNodes,           
            # 40 epochs, 10 batchSize, 10 k_fold_split

# Standardized: mean is -48.66, std is 18.17
            # running model with 29 firstLayerNodes, 44 secondLayerNodes,           
            # 40 epochs, 5 batchSize, 5 k_fold_split

# Standardized: mean is -51.15, std is 14.19, 
            # running model with 6 firstLayerNodes, 50 secondLayerNodes, 
            # 30 epochs, 20 batchSize, 10 k_fold_split
        
# Dropout after first dense layer
